In [1]:
import Model as md
import Resnet as res
import data as dta

import torch
import torch.nn as nn
import torch.optim as optim
from pytorch_metric_learning import losses
from torchvision import transforms as T
from torch.utils.data import DataLoader
from absl import app
from absl import flags
from absl import logging
import os
from tqdm import tqdm

from torch.cuda import amp
import pandas as pd
import math

In [2]:
encoder = res.ResNet50().cuda()
proj_head = md.Projection_Head(encoder.representation_dim, 1).cuda()
sup_head = md.Supervised_Head(proj_head.output, 5).cuda()

In [3]:
optimizer = optim.SGD(list(encoder.parameters()) + list(proj_head.parameters()) +
                      list(sup_head.parameters()), lr=0.004, weight_decay=1e-6,momentum=0.9)

cross_entropy_loss = nn.CrossEntropyLoss()

In [4]:
#Transformations
transf = T.Compose([
    T.CenterCrop(400),
    T.Resize(112),
    T.RandomVerticalFlip(),
    T.RandomHorizontalFlip(),
    T.RandomGrayscale(p=0.2),
    T.ToTensor(),
])

transf_eval = T.Compose([
    T.CenterCrop(400),
    T.Resize(112),
    T.ToTensor(),
])

In [5]:
astro_ds = dta.AstroDataset('nair_unbalanced_train.csv', 'imagenes_clasificadas_nair/', transform=transf)
dataset_astro = DataLoader(astro_ds,batch_size=128, shuffle=True,num_workers=6)

In [6]:
eval_astro_ds = dta.AstroDataset('nair_unbalanced_train.csv', 'imagenes_clasificadas_nair/', transform=transf_eval)
dataset_astro_val = DataLoader(eval_astro_ds,batch_size=128, shuffle=False,num_workers=6)

In [7]:
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, (astro_ds.__len__() * 5 // 128 + 1))
scaler = amp.GradScaler()

In [8]:
def save_model(encoder, projection_head, sup_head, epoch_number, optimizer, scheduler):
    torch.save({
        'encoder': encoder.state_dict(),
        'projection_head': projection_head.state_dict(),
        'supervised_head': sup_head.state_dict(),
        'epoch': epoch,
        'optimizer': optimizer.state_dict(),
        'scheduler': scheduler.state_dict()
    }, 'model/model.pt')

In [9]:
df = pd.DataFrame(columns=['Epoch','SupervisedLoss','Accuracy'])

In [10]:
checkpoint = torch.load('model/model.pt')
encoder.load_state_dict(checkpoint['encoder'])
proj_head.load_state_dict(checkpoint['projection_head'], strict=False)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['second_layer.weight', 'second_layer.bias', 'second_bn.weight', 'second_bn.bias', 'second_bn.running_mean', 'second_bn.running_var', 'second_bn.num_batches_tracked', 'third_layer.weight', 'third_bn.running_mean', 'third_bn.running_var', 'third_bn.num_batches_tracked'])

In [11]:
for epoch in range(0, 5):
    acc_epoch = 0
    epoch_loss = 0
    m=0
    #use tqdm
    tqdm_loop = tqdm(enumerate(dataset_astro), total=len(dataset_astro), leave=True)
    #tqdm_loop_val = tqdm(enumerate(dataset_astro_val), total=len(dataset_astro_val), leave=True)
    encoder.train()
    proj_head.train()
    sup_head.train()
    for batch_idx, data in tqdm_loop:
        image = data[0].cuda()
        labels = data[1].cuda()
        optimizer.zero_grad()
        with amp.autocast():
            output = sup_head(proj_head(encoder(image)))
            #print(output.shape)
            #print(torch.sum(output, 0))
            loss = cross_entropy_loss(output, labels)
            epoch_loss += loss.item()

        scaler.scale(loss).backward()
        #loss.backward()
        scaler.step(optimizer)
        scaler.update()
        #optimizer.step()
        #epoch_loss += 
        #print(loss.item())
        scheduler.step()
        #update progress bar
        tqdm_loop.set_description(f'Epoch [{epoch}/{5}]')
        tqdm_loop.set_postfix(loss = loss.item())
        
        with torch.no_grad():
            softmax = nn.Softmax(dim=1)
            output = softmax(output)
            #print(output)
            acc = torch.argmax(output,dim=1)
            acc = (acc == labels).float().sum()
            acc_epoch += acc
    #save_model(encoder, proj_head, epoch, optimizer, scheduler)
    acc_epoch /= astro_ds.__len__()
    epoch_loss /= (batch_idx+1)
    d_list = [epoch, epoch_loss, acc_epoch.item()]
    df.loc[len(df), :] = d_list
    print('Epoch: {}, Loss: {}, Contrastive Accuracy: {}'.format(epoch, epoch_loss, acc_epoch*100))
    
    encoder.eval()
    proj_head.eval()
    sup_head.eval()
    acc_vall, loss_val = 0 , 0
    with torch.no_grad():
        for image, label in dataset_astro_val:
        #for batch_idx, data in tqdm_loop_val:
            image = image.cuda()
            label = label.cuda()
            with amp.autocast():
                output_val = sup_head(proj_head(encoder(image)))
                #print(output.shape)
                #print(torch.sum(output, 0))
                losss = cross_entropy_loss(output_val, label)
                #print(losss)
                loss_val += losss.item()
                output_val = softmax(output_val)
                acc_val = torch.argmax(output_val,dim=1)
                acc_val = (acc_val == label).float().sum()
                acc_vall += acc_val
                #update progress bar
            #tqdm_loop_val.set_description(f'Epoch [{epoch}/{5}]')
            #tqdm_loop_val.set_postfix(loss = loss.item())
        acc_vall /= eval_astro_ds.__len__()
        loss_val /= eval_astro_ds.__len__()
    print('Validation Step at Epoch: {}, Loss: {}, Sup Accuracy: {}'.format(epoch, loss_val, acc_vall*100))

Epoch [0/5]: 100%|██████████████████████████████████████████████████████████| 74/74 [00:18<00:00,  4.09it/s, loss=1.78]


Epoch: 0, Loss: 1.1395058213053524, Contrastive Accuracy: 54.88729476928711
Validation Step at Epoch: 0, Loss: nan, Sup Accuracy: 59.11761474609375


Epoch [1/5]: 100%|█████████████████████████████████████████████████████████| 74/74 [00:17<00:00,  4.30it/s, loss=0.969]


Epoch: 1, Loss: 1.0254793650395162, Contrastive Accuracy: 59.67311096191406
Validation Step at Epoch: 1, Loss: nan, Sup Accuracy: 61.26482009887695


Epoch [2/5]: 100%|██████████████████████████████████████████████████████████| 74/74 [00:17<00:00,  4.24it/s, loss=1.33]


Epoch: 2, Loss: 0.9536526501178741, Contrastive Accuracy: 60.65591049194336
Validation Step at Epoch: 2, Loss: nan, Sup Accuracy: 61.47846984863281


Epoch [3/5]: 100%|█████████████████████████████████████████████████████████| 74/74 [00:17<00:00,  4.21it/s, loss=0.749]


Epoch: 3, Loss: 0.9312415461282473, Contrastive Accuracy: 60.64522933959961
Validation Step at Epoch: 3, Loss: nan, Sup Accuracy: 61.89509582519531


Epoch [4/5]: 100%|█████████████████████████████████████████████████████████| 74/74 [00:17<00:00,  4.29it/s, loss=0.969]


Epoch: 4, Loss: 0.9241772616231764, Contrastive Accuracy: 60.24997329711914
Validation Step at Epoch: 4, Loss: nan, Sup Accuracy: 61.809635162353516


In [12]:
df

,Epoch,SupervisedLoss,Accuracy
0,0,1.139506,0.548873
1,1,1.025479,0.596731
2,2,0.953653,0.606559
3,3,0.931242,0.606452
4,4,0.924177,0.6025


In [13]:
a = None
if a is not None:
    print('hello')

In [14]:
d = 0 if a is not None else 4

In [15]:
s = None

In [16]:
list(s.parameters())

AttributeError: 'NoneType' object has no attribute 'parameters'